# Sobre o projeto

Este projeto tem como objetivo a implementação de diversas soluções para o problema VRP.

Para executá-lo:
1. Vá para a sessão de geração de instancias do problema para gerar arquivos de grafos com o tamanho e probabilidade que desejar

2. O projeto está dividido em diferentes sessões com o nome de cada implamentação, érecomentado seguir a ordem do notebook, as instruções de execução se encontram na primeira iplementação (a global) e são as mesmas para todas a implementações, atenção somente aos nomes dos arquivos.

# Projeto SuperComputação 2024.1 (Enunciado)

## Otimização de Rotas de Veículos (_Vehicle Routing Problem_ - VRP)

**Objetivo:**

Desenvolver uma aplicação em C++ que otimize as rotas de uma frota de veículos destinados à entrega de produtos a diversos clientes, buscando minimizar o custo total das rotas.


**Descrição:**

Você está encarregado de criar uma solução computacional para o problema de Otimização de Rotas de Veículos (VRP) para uma empresa de logística. A empresa possui uma frota de veículos que são usados para entregar produtos a uma série de clientes. Cada veículo tem uma capacidade de carga máxima, e cada cliente tem uma demanda específica que deve ser atendida. O objetivo é determinar as rotas ótimas para os veículos, de forma que todos os clientes sejam atendidos, respeitando as restrições de capacidade dos veículos, e minimizando o custo total das entregas.


**Restrições e Requisitos:**
1. **Capacidade dos Veículos:** Cada veículo tem uma capacidade máxima de carga que não pode ser excedida.
2. **Demanda dos Clientes:** Cada cliente tem uma demanda específica que deve ser completamente atendida por um único veículo.
3. **Custo da Rota:** Cada rota possui um custo, que simboliza o cômputo agregado da distância percorrida, do tempo de entrega, e do consumo de combustível. O objetivo é minimizar o custo total.
4. **Número máximo de visitas por rota:** Cada rota pode visitar um número máximo de cidades.
5. **Ponto de Partida e Chegada:** Todos os veículos começam e terminam suas rotas no depósito da empresa.


**Desafios Computacionais:**

- O VRP é um problema NP-difícil, o que significa que sua complexidade computacional cresce exponencialmente com o aumento do número de clientes e veículos.
- A busca exaustiva por todas as possíveis combinações de rotas torna-se impraticável para instâncias maiores do problema.


**Composição da Solução:**
1. Implementação de um algoritmo de _busca global_ para o problema (força bruta);
2. Implementação de alguma solução aproximada (_heurística_ ou _busca local_);
3. Implementação de 2 soluções paralelizadas, sendo:

  a. _Paralelização com Threads OpenMP_: Utilizar threads e a biblioteca OpenMP para paralelizar o algoritmo desenvolvido, com o objetivo de reduzir o tempo de execução. Analisar e implementar a paralelização de componentes do algoritmo que possam se beneficiar da execução concorrente, como a geração de soluções iniciais ou a busca local.

  b. _Paralelização e Distribuição do Processamento com MPI_: Implementar uma versão do algoritmo que utilize a interface de passagem de mensagens MPI para distribuir o processo de busca de soluções em múltiplos processos, possivelmente executando em diferentes nós de um cluster. A estratégia deve permitir a exploração paralela do espaço de soluções e a troca eficiente de informações entre os processos.



**Entregáveis:**

1. **Código Fonte:** Código fonte em C++ comentado e organizado.
2. **Relatório:** Um relatório descrevendo a abordagem utilizada, incluindo descrição das heurísticas e métodos de busca local, bem como as estratégias de paralelização adotadas.
3. **Análise de Desempenho:** Uma análise de desempenho da solução proposta, incluindo tempos de execução e qualidade das soluções encontradas, com base em instâncias de teste de diferentes tamanhos.
4. **Instruções de Uso:** Breve documentação sobre como compilar e executar a aplicação, incluindo exemplos de uso.


**DICA**:
Não deixe de estudar os exemplos deste link https://vrpy.readthedocs.io/en/master/examples.html, para entender como gerar a avaliar soluções para o problema. Compare sua solução com a solução desta biblioteca!

# GERAÇÃO DE INSTÂNCIAS DO PROBLEMA

Execute o código abaixo algumas vezes gerando instâncias de tamanhos e complexidades distintas. Você deve experimentar alterações de:
- **num_nos**: número de cidades a serem visitados;
- **probabilidade**: a probabilidade de ser criada uma rota direta entre 2 cidades;


In [1]:
import random
# from networkx import set_node_attributes

def gerar_dicionario_demandas(N):
    """
    Gera um dicionário onde a chave é um int de 1 até N e o valor é um inteiro aleatório de 1 até 10.

    :param N: Número máximo para as chaves do dicionário.
    :return: Dicionário com chaves de 1 até N e valores inteiros aleatórios de 1 até 10.
    """
    return {i: random.randint(1, 10) for i in range(1, N)}


def gerar_entradas_grafo(num_nos, max_peso=100, probabilidade=0.125):
    """
    Gera um grafo para o problema de otimização de rotas de veículos.

    :param num_nos: Número de nós no grafo, incluindo o depósito.
    :param max_peso: Peso máximo para as arestas do grafo.
    :param probabilidade: Probabilidade de criar uma rota entre duas cidades.
    :return: Um dicionário representando o grafo onde as chaves são tuplas representando as arestas (nó1, nó2)
             e os valores são os pesos dessas arestas.
    """
    grafo = {}
    # Gerar pesos para arestas entre o depósito e outros nós
    for i in range(1, num_nos):
        grafo[(0, i)] = random.randint(1, max_peso)
        grafo[(i, 0)] = grafo[(0, i)]  # Assume que a distância de volta ao depósito é a mesma

    # Gerar pesos para arestas entre todos os outros pares de nós
    for i in range(1, num_nos):
        for j in range(i+1, num_nos):
            if random.random() > (1 - probabilidade):  # Verifica a probabilidade
                peso = random.randint(1, max_peso)
                grafo[(i, j)] = peso

    return grafo

############################################
#             Exemplo de uso
############################################
num_nos = 1000                                   # Número total de nós incluindo o depósito
probabilidade = 0.125                           # Probabilidade de criar uma rota entre duas cidades
nome_arquivo = "grafo_"+str(num_nos)+"_"+str(probabilidade)+".txt"  # Nome do arquivo onde será salvo o grafo

def generator(num_nos, probabilidade, nome_arquivo):

    demandas = gerar_dicionario_demandas(num_nos)  # Gera as demandas para cada nó
    grafo = gerar_entradas_grafo(num_nos, probabilidade=probabilidade)          # Gera o grafo que representa os locais e custos entre eles

    # Salva o grafo em um arquivo TXT
    with open(nome_arquivo, 'w') as arquivo:
        arquivo.write(str(num_nos) + "\n")    # Número de nós, incluindo depósito
        for local, demanda in demandas.items():
            linha = f"{local} {demanda}\n"      # Par LOCAL DEMANDA
            arquivo.write(linha)

        arquivo.write(str(len(grafo)) + "\n") # Número de arestas
        for aresta, peso in grafo.items():
            linha = f"{aresta[0]} {aresta[1]} {peso}\n" # Trio: ORIGEM DESTINO CUSTO
            arquivo.write(linha)

        print(f"Grafo salvo em {nome_arquivo}")


In [6]:
num_nos = 10                                   # Número total de nós incluindo o depósito
probabilidade = 0.125                           # Probabilidade de criar uma rota entre duas cidades
nome_arquivo = "grafo_"+str(num_nos)+"_"+str(probabilidade)+".txt"  # Nome do arquivo onde será salvo o grafo

generator(num_nos, probabilidade, nome_arquivo)

Grafo salvo em grafo_10_0.125.txt


# GERAÇÃO DE SOLUÇÕES (testes de validação com VRPy) (se desejar validar)

Instale a biblioteca [VRPy](https://vrpy.readthedocs.io/en/latest/getting_started.html) para resolver problemas de otimização de rotas.

Para cada instância do problema gerada, execute este código para gerar a solução esperada.

Não deixe de estudar a documentação da biblioteca! Especialmente este link: https://vrpy.readthedocs.io/en/latest/examples.html#a-simple-example

Aqui você pode gerar soluções diferentes alterando:
- **load_capacity**: a quantidade e capacidade de veículos;
- **num_stops**: o número máximo de paradas numa rota;

In [ ]:
!pip install vrpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.0/140.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 56.1 MB/s eta 0:00:00
  Created wheel for vrpy: filename=vrpy-0.5.1-py3-none-any.whl size=177559 sha256=a1ab4dddb6e1afed632422625d8ce86233c0e426137bb26a46bb14bd3be4c311
  Stored in directory: /root/.cache/pip/wheels/ed/82/02/25ff2a363ada1eabb60760332578d6c457a4224d4656900947
Successfully built vrpy


In [ ]:
from networkx import DiGraph, set_node_attributes
from vrpy import VehicleRoutingProblem


def ler_arquivo_grafo(caminho_arquivo):
    with open(caminho_arquivo, 'r') as arquivo:
        # Lê o número de nós
        N = int(arquivo.readline().strip())-1

        # Lê as demandas dos nós
        demandas = {}
        for _ in range(N):
            linha = arquivo.readline().strip().split()
            id_no, demanda = int(linha[0]), int(linha[1])
            demandas[id_no] = demanda

        # Lê o número de arestas
        K = int(arquivo.readline().strip())

        # Lê as arestas
        arestas = []
        for _ in range(K):
            linha = arquivo.readline().strip().split()
            origem, destino, peso = int(linha[0]), int(linha[1]), int(linha[2])
            arestas.append((origem, destino, peso))

    return demandas, arestas

############################################
#             Usando a funcao
############################################
caminho_arquivo = 'grafo_500_0.125.txt'
demandas, arestas = ler_arquivo_grafo(caminho_arquivo)


G = DiGraph()
for inicio, fim, custo in arestas:
    if inicio==0: inicio="Source"
    if fim==0: fim="Sink"
    G.add_edge(inicio, fim, cost=custo)

set_node_attributes(G, values=demandas, name="demand")

prob = VehicleRoutingProblem(G, load_capacity=15)  # Pode alterar a capacidade
prob.num_stops = 5                                 # Pode alterar o número máximo de paradas
prob.solve()

In [ ]:
prob.best_routes

{1: ['Source', 2, 9, 'Sink'],
 2: ['Source', 5, 'Sink'],
 3: ['Source', 7, 'Sink'],
 4: ['Source', 1, 8, 'Sink'],
 5: ['Source', 4, 6, 'Sink'],
 6: ['Source', 3, 'Sink']}

In [ ]:
prob.best_value

741

In [ ]:
prob.best_routes_cost

{1: 167, 2: 24, 3: 74, 4: 205, 5: 223, 6: 48}

In [ ]:
prob.best_routes_load

{1: 11, 2: 4, 3: 7, 4: 11, 5: 15, 6: 3}

# Algoritmo de busca global (Pseudocódigo) e Rubrica



```cpp
Função ResolverVRPComDemanda(Locais, Demanda, C):
    // Locais: Lista de locais para entrega (excluindo o depósito)
    // Demanda: Dicionário mapeando cada local à sua demanda
    // C: Capacidade do veículo
    
    MelhorRota = NULL
    MenorCusto = INFINITO

    // Gera todas as combinações possíveis de rotas considerando a capacidade do veículo
    Combinações = GerarTodasAsCombinações(Locais)
    
    Para cada combinação em Combinações faça:
        Se VerificarCapacidade(combinação, Demanda, C) então:
            CustoAtual = CalcularCusto(combinação)
            Se CustoAtual < MenorCusto então:
                MenorCusto = CustoAtual
                MelhorRota = combinação
                
    Retornar MelhorRota, MenorCusto

Função GerarTodasAsCombinações(Locais):
    // Gera todas as permutações possíveis de locais e agrupa em rotas válidas conforme a capacidade
    // Esta função é bastante complexa, pois precisa considerar todas as subdivisões possíveis dos locais em rotas que atendam à capacidade do veículo
    // Retorna uma lista de combinações válidas
    Retornar combinações

Função VerificarCapacidade(Rota, Demanda, C):
    // Verifica se a demanda total da rota não excede a capacidade do veículo
    CargaTotal = 0
    Para cada local em Rota faça:
        CargaTotal += Demanda[local]
    Se CargaTotal > C então:
        Retornar Falso
    Senão:
        Retornar Verdadeiro

Função CalcularCusto(Rota):
    // Calcula o custo de uma rota com base na distância, tempo ou outro critério
    // O custo pode depender de fatores como a distância total percorrida, o número de veículos necessários, entre outros
    Retornar custo

// Início do programa
// Define os locais, suas demandas e a capacidade do veículo
Locais = [...]
Demanda = {...}
Capacidade = C

MelhorRota, MenorCusto = ResolverVRPComDemanda(Locais, Demanda, Capacidade)
Exibir "A melhor rota é:", MelhorRota, "com custo total de:", MenorCusto
```

## Critérios de avaliação

A correção do projeto levará em conta:

+ [Até 1 pontos] **Organização geral**
  - Organização do código fonte e/ou do notebook que centraliza seus códigos. Se a entrega for via jupyter notebook, ele deve conter células markdown que guiem a apreciação do trabalho. Caso seu código esteja organizado em arquivos ".cpp", ".py", etc, submeta também um relatório em PDF descrevendo seu trabalho;

+ [Até 4 pontos] **Implementações**
  - Sua entrega deve conter ao menos 3 implementações (uma para cada solução: exaustiva, OpenMP e MPI). Soluções extras nestas vertentes são encorajadas e serão valorizadas;
  - A parte de MPI deve ser obrigatoriamente executada no cluster. Portanto, submeta também os arquivos de configuração dos executáveis em batch;
  - Esperamos códigos seguindo as boas práticas de implementação e devidamente comentados;

+ [Até 3 pontos] **Avaliação de resultados**
  - Comparações das abordagens com tamanhos diferentes de grafos;
  - Clareza na comunicação dos resultados. Ex: gráficos e/ou pequenos textos que descrevem os resultados obtidos;
  - Justificativas: é esperado que a execução paralela seja mais rápida que a sequencial, assim como uma heurística seja mais rápida que a abordagem exaustiva. Caso seus resultados diferem do esperado, argumente potenciais causas;

+ [Até 1 ponto] **Extras**
  - A rubrica não cita o máximo de implementações a ser feita. Caso você opte por fazer algo além das 4 solicitadas, daremos até 1 ponto extra na nota do trabalho pelo esforço;




# GLOBAL

Como primeiro passo deste projeto buscamos realizar o metodo da busca global pela melhor solução, onde são realizadas todas as cobinações possíveis de caminhos e através desta busca exaustiva encontramos o melhor resultado possível para o problema.

Esta solução é inviável na grande maioria dos casos, uma vez que o tempo de execução cresce exponencialmente com a entrada, sendo possível executá-lo em tempo aceitável apenas em situações com algumas dezenos de nós para o vrp

Apesar da intratabilidade para grandes entradas, enta pode ainda ser uma solução viável se:

1. as entradas forem sempre pequenas
2. é necessário que sempre se obtenha a melhor solução possível


In [5]:
%%file global.cpp

#include <iostream>  // For input and output
#include <vector>    // For using vectors
#include <algorithm> // For using algorithms like next_permutation
#include <fstream>   // For file handling
#include <set>       // For using sets
#include <limits>    // For numeric limits
#include <chrono>    // For measuring execution time
#include <numeric>   // For numeric operations like iota

using namespace std;
using namespace std::chrono;

// Function to read demands from a file and store them in a vector
vector<int> ReadDemands(const string& fileName, int& numVerts) {
    ifstream file(fileName); // Open the file
    if (!file.is_open()) {   // Check if the file is open
        cerr << "Error opening file " << fileName << endl;
        exit(1);
    }

    file >> numVerts; // Read the number of vertices

    vector<int> demands(numVerts, 0); // Initialize the demands vector
    int dest, demand;
    for (int i = 1; i < numVerts; i++) {
        file >> dest >> demand; // Read destination and demand
        demands[dest] = demand; // Store the demand in the vector
    }
    file.close(); // Close the file
    return demands;
}

// Function to read routes from a file and store them in a 2D vector
vector<vector<int>> ReadRoutes(const string& fileName, int& numVerts) {
    ifstream file(fileName); // Open the file
    if (!file.is_open()) {   // Check if the file is open
        cerr << "Error opening file " << fileName << endl;
        exit(1);
    }

    file >> numVerts; // Read the number of vertices

    // Skip the initial routes data as it's not used
    for (int i = 0; i < numVerts - 1; i++) {
        int x, y;
        file >> x >> y;
    }

    int numRoutes;
    file >> numRoutes; // Read the number of routes

    // Initialize the routes matrix with zeros
    vector<vector<int>> routes(numVerts, vector<int>(numVerts, 0));

    int src, dest, cost;
    while (file >> src >> dest >> cost) {
        routes[src][dest] = cost; // Store the cost in the matrix
    }
    file.close(); // Close the file
    return routes;
}

// Function to calculate the cost of a given route
int CalcCost(const vector<int>& route, const vector<vector<int>>& routes) {
    int cost = 0;
    for (size_t i = 0; i < route.size() - 1; ++i) {
        int src = route[i];
        int dest = route[i + 1];
        if (routes[src][dest] == 0) { // If there's no direct route
            return -1;
        }
        cost += routes[src][dest]; // Add the cost of the segment
    }
    return cost;
}

// Function to check if a route's total demand is within capacity
bool CheckCapacity(const vector<int>& route, const vector<int>& demands, int capacity) {
    int totalLoad = 0;
    for (int idx : route) {
        totalLoad += demands[idx]; // Sum the demands of all nodes in the route
    }
    return totalLoad <= capacity; // Check if within capacity
}

// Function to check if a route exists in a set of valid combinations
bool RouteExists(const vector<int>& sub, const vector<vector<int>>& validCombs) {
    return find(validCombs.begin(), validCombs.end(), sub) != validCombs.end();
}

// Function to generate all valid combinations of routes
void GenerateAllCombs(const vector<vector<int>>& routes, int routeSize, vector<vector<int>>& results,
                      const vector<int>& demands, int capacity) {
    int n = routes.size();
    vector<int> indices(n);
    iota(indices.begin(), indices.end(), 0); // Fill indices with 0, 1, ..., n-1

    // Generate all permutations of routes
    while (next_permutation(indices.begin() + 1, indices.end())) {
        for (int i = 0; i < n; i += routeSize) {
            int j = min(i + routeSize, n);
            vector<int> sub(indices.begin() + i, indices.begin() + j);
            sub.insert(sub.begin(), 0); // Add starting point
            sub.push_back(0);           // Add ending point
            // Check if the route is valid
            if (CalcCost(sub, routes) != -1 && !RouteExists(sub, results) && CheckCapacity(sub, demands, capacity)) {
                results.push_back(sub); // Add to results if valid
            }
        }
    }
}

// Function to check if there are no overlapping nodes in different routes
bool NoOverlap(const vector<vector<int>>& routes, int totalNodes) {
    set<int> visited;
    for (const auto& route : routes) {
        for (int node : route) {
            if (node == 0) continue; // Skip the depot
            if (visited.find(node) != visited.end()) { // If node already visited
                return false;
            }
            visited.insert(node); // Mark the node as visited
        }
    }
    return visited.size() == static_cast<size_t>(totalNodes) - 1;
}

// Recursive function to generate all combinations of routes
void GenerateCombsRec(const vector<vector<int>>& routes, vector<int>& currComb, vector<vector<int>>& allCombs,
                      int combSize, int numVerts, int start) {
    if (currComb.size() == combSize) {
        vector<vector<int>> realComb;
        for (size_t idx : currComb) {
            realComb.push_back(routes[idx]);
        }
        if (NoOverlap(realComb, numVerts)) {
            allCombs.push_back(currComb); // Add to allCombs if no overlap
        }
        return;
    }

    for (size_t i = start; i < routes.size(); i++) {
        currComb.push_back(i); // Add route index to current combination
        GenerateCombsRec(routes, currComb, allCombs, combSize, numVerts, i + 1);
        currComb.pop_back(); // Backtrack
    }
}

// Wrapper function to generate all route combinations
vector<vector<int>> GenerateAllCombs(const vector<vector<int>>& routes, int numVerts) {
    vector<vector<int>> allCombs;
    vector<int> currComb;

    for (size_t k = 1; k <= routes.size(); k++) {
        GenerateCombsRec(routes, currComb, allCombs, k, numVerts, 0);
    }

    return allCombs;
}

// Function to calculate the total cost of a combination of routes
int TotalCost(const vector<int>& comb, const vector<vector<int>>& possRoutes, const vector<vector<int>>& routes) {
    int totalCost = 0;
    for (const auto& routeIdx : comb) {
        int routeCost = CalcCost(possRoutes[routeIdx], routes);
        if (routeCost == -1) return -1; // If any route is invalid
        totalCost += routeCost; // Add route cost to total cost
    }
    return totalCost;
}

// Function to find the combination of routes with the minimum cost
int MinCost(const vector<vector<int>>& allCombs, const vector<vector<int>>& possRoutes, const vector<vector<int>>& routes) {
    int minCost = numeric_limits<int>::max();
    vector<int> minCostIndices;

    for (const auto& comb : allCombs) {
        int totalCost = TotalCost(comb, possRoutes, routes);
        if (totalCost != -1 && totalCost < minCost) {
            minCost = totalCost;
            minCostIndices = comb; // Update minimum cost and indices
        }
    }

    cout << "Minimum cost combination: ";
    cout << minCost << endl; // Print minimum cost

    return minCost;
}

// Function to solve the Vehicle Routing Problem (VRP)
int SolveVRP(const vector<vector<int>>& locations, const vector<int>& demands, int capacity, int numVerts, int routeSize, int maxStops) {
    vector<vector<int>> possRoutes;

    // Generate possible routes
    for (int i = 1; i < routeSize; i++) {
        GenerateAllCombs(locations, i, possRoutes, demands, capacity);
    }

    // Generate all combinations of possible routes
    vector<vector<int>> allCombs = GenerateAllCombs(possRoutes, numVerts);
    MinCost(allCombs, possRoutes, locations); // Find the minimum cost combination

    return 0;
}

// Main function to execute the program
int main(int argc, char* argv[]) {
    if (argc < 2) {
        cerr << "Usage: " << argv[0] << " <input file>" << endl;
        return 1;
    }
    string fileName = argv[1];
    int capacity = 15;
    int numVerts;
    int routeSize = 3;
    int maxStops = 5;

    // Read demands and locations from the input file
    vector<int> demands = ReadDemands(fileName, numVerts);
    vector<vector<int>> locations = ReadRoutes(fileName, numVerts);

    auto start = high_resolution_clock::now(); // Start measuring execution time

    SolveVRP(locations, demands, capacity, numVerts, routeSize, maxStops); // Solve the VRP

    auto end = high_resolution_clock::now(); // End measuring execution time
    auto duration = duration_cast<milliseconds>(end - start).count();
    cout << "Execution time: " << duration << " ms" << endl; // Print execution time

    return 0;
}

Writing global.cpp


## Execução

Nas células abiaxo está construído um padrão que também será seguido para as próximas implementações, definido pelos seguintes passos.

1. Definição do arquivo de entrada, não se esqueça de gerar os qrquivos anteriormente na etapa de geração de entradas
2. compilação e execução, caso deseje executar no colab ou notebook é necessário apenas rodas as células abaixo, caso deseje fazer isso no cluster, o arquivo slurm está disponibilizado na seção slurm abaixo e deve ser rodado com o comando sbatch.

In [7]:
arquivo = "grafo_10_0.125.txt"

In [8]:
!g++ -g -Wall -o global global.cpp

global.cpp: In function ‘void GenerateCombsRec(const std::vector<std::vector<int> >&, std::vector<int>&, std::vector<std::vector<int> >&, int, int, int)’:
global.cpp:132:25: warning: comparison of integer expressions of different signedness: ‘std::vector<int>::size_type’ {aka ‘long unsigned int’} and ‘int’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wsign-compare-Wsign-compare]8;;]
  132 |     if (currComb.size() == combSize) {
      |         ~~~~~~~~~~~~~~~~^~~~~~~~~~~


In [9]:
!./global $arquivo

Minimum cost combination: 913
Execution time: 6652 ms


In [ ]:
arquivo = "grafo_20_0.125.txt"
!./global $arquivo

### Slurm


In [ ]:
%%file global.slurm

#!/bin/bash
#SBATCH --job-name=global
#SBATCH --nodes=1
#SBATCH --partition=express
#SBATCH --mem=500M

./global $arquivo

Writing global.slurm


## Resultados


Para a analize desta conclusão o arquivo foi executado no colab conforme descrito na seção anterior.

Ao executar o problema com entradas de 10 ou 20 nós ja é possível perceber o grau de complexidade do problema, sendo que ao rodá-lo para 20 nós já possuimos um tempo de execução drásticamente maior (levam-se alguns vários minutos) do para 10 nós (leva um pouco mais de 7 segundos) e para entradas de 50 ou 100 ja se torna intratável.

Isso mostra que a solução implementada seria inutilizável em um problema real de traçado de rotas, sendo necessária a execução de outra abordagem para reduzir o tempo de processamento.

# Heuristica

Com o objetivo de se tentar alcançar uma implementação viável para o mundo real, o próximo passo será o de se aplicar a heurítica de de Clarke e Wright.

Essa heurística foi escolhida uma vez que ela é simples de se entendere implementar comparada a outras heurísticas do VRP e apresenta resultados bons e rápidos com frequência.

Os passos desta heurística são os seguintes:

1. Começa-se com uma solução inicial onde cada cliente é atendido individualmente por um veículo que vai do depósito até o cliente e retorna diretamente ao depósito. Ou seja, se houver n clientes, teremos n rotas inicialmente.

2. Para cada par de clientes 𝑖 e 𝑗, calcula-se a economia 𝑆𝑖𝑗
associada ao combinar os dois clientes em uma única rota, em vez de atender a cada um individualmente. esta operação é representada pela fórmula:
𝑆𝑖𝑗 = d0i+d0j−dij

3. As economias 𝑆𝑖𝑗 são ordenadas de forma decrescente.

4. Inicia-se a construção das rotas. Inicialmente, cada cliente está em uma rota separada. A heurística começa a combinar rotas baseando-se nas economias calculadas, começando pela maior economia.
Ao combinar duas rotas que incluem os clientes 𝑖 e 𝑗, verifica-se se é possível juntar essas rotas sem violar nenhuma restrição do problema (por exemplo, capacidade dos veículos ou restrições de tempo).
A combinação é feita unindo a rota que termina em 𝑖 com a rota que começa em 𝑗.

5. Começamos a iterar até que não seja possível mais combinar rotas de forma benéfica ou até que todas as economias possíveis tenham sido avaliadas.



In [10]:
%%file heuristica.cpp


#include <iostream>  // For input and output operations
#include <vector>    // For using vectors
#include <algorithm> // For using algorithms like sort
#include <map>       // For using maps
#include <climits>   // For integer limits
#include <fstream>   // For file handling
#include <set>       // For using sets
#include <limits>    // For numeric limits
#include <chrono>    // For measuring execution time

using namespace std;
using namespace std::chrono;

// Function to read the input file and return a vector with demands
vector<int> LerDestinoDemanda(const string& nomeArquivo, int& numVertices) {
    ifstream arquivo(nomeArquivo); // Open the file
    if (!arquivo.is_open()) {      // Check if the file is open
        cerr << "Erro ao abrir o arquivo " << nomeArquivo << endl;
        exit(1);
    }

    arquivo >> numVertices; // Read the number of vertices

    vector<int> grafo(numVertices, 0); // Initialize the graph with zero demands
    int destino, demanda;
    for (int i = 1; i < numVertices; i++) {
        arquivo >> destino >> demanda; // Read destination and demand
        grafo[destino] = demanda;      // Store the demand in the graph
    }
    arquivo.close(); // Close the file
    return grafo;
}

// Function to read the input file and return a vector with possible routes
vector<vector<int>> LerRotasPossiveis(const string& nomeArquivo, int& numVertices) {
    ifstream arquivo(nomeArquivo); // Open the file
    if (!arquivo.is_open()) {      // Check if the file is open
        cerr << "Erro ao abrir o arquivo " << nomeArquivo << endl;
        exit(1);
    }
    // Read the first line of the file to get the number of vertices
    arquivo >> numVertices;

    // Skip the next numVertices-1 lines
    for (int i = 0; i < numVertices-1; i++) {
        int x, y;
        arquivo >> x >> y;
    }

    // Read the number of possible routes
    int numRotas;
    arquivo >> numRotas;

    // Create a graph with numVertices vertices and initialize with 0, then read the routes
    vector<vector<int>> grafo(numVertices, vector<int>(numVertices, 0));

    int origem, destino, custo;
    while (arquivo >> origem >> destino >> custo) {
        grafo[origem][destino] = custo; // Store the route cost (directed graph)
    }
    arquivo.close(); // Close the file
    return grafo;
}

// Struct to store the economy values for the Clarke-Wright algorithm
struct Economia {
    int i, j;     // Nodes
    double valor; // Economy value
    Economia(int i, int j, double valor) : i(i), j(j), valor(valor) {} // Constructor
};

// Function to compare economy values
bool compararEconomias(const Economia& a, const Economia& b) {
    return a.valor > b.valor;
}

// Function to find the route of a node
int encontrarRota(int node, const vector<vector<int>>& rotas, const vector<vector<int>>& distancias) {
    for (size_t i = 0; i < rotas.size(); ++i) {
        auto it = find(rotas[i].begin(), rotas[i].end(), node);
        if (it != rotas[i].end()) {
            // Check if the route has a cost of 0
            for (size_t j = 0; j < rotas[i].size() - 1; ++j) {
                if (distancias[rotas[i][j]][rotas[i][j + 1]] == 0) {
                    return -1; // Invalid route
                }
            }
            return i;
        }
    }
    return -1;
}

// Function to calculate the cost of a route
int calcularCustoRota(const vector<int>& rota, const vector<vector<int>>& distancias) {
    int custo = 0;
    for (size_t i = 0; i < rota.size() - 1; ++i) {
        custo += distancias[rota[i]][rota[i + 1]];
    }
    return custo;
}

// Function to combine routes based on economies
void CombinaRotas(vector<Economia>& economias, vector<vector<int>>& rotas, const vector<vector<int>>& distancias, const vector<int>& demandas, int maxParadas, int capacidade);

// Function to verify if a new route is valid
void VerificaValida(vector<int>& novaRota, const vector<vector<int>>& distancias, bool& rotaValida, vector<vector<int>>& rotas, int rotaI, int rotaJ);

// Function to calculate the total cost of all routes
void CalcCustoTotal(vector<vector<int>>& rotas, const vector<vector<int>>& distancias);

// Function to implement the Clarke-Wright heuristic
/**
 * Applies the Clarke-Wright heuristic algorithm to solve a vehicle routing problem.
 *
 * @param distancias A 2D vector representing the distances between each pair of nodes.
 * @param capacidade The maximum capacity of each vehicle.
 * @param demandas A vector representing the demand of each customer node.
 * @param maxParadas The maximum number of stops allowed in each route.
 */
void clarkeWright(const vector<vector<int>>& distancias, int capacidade, const vector<int>& demandas, int maxParadas) {
    int n = distancias.size() - 1; // Number of customers (excluding depot)

    // Initialize individual routes
    vector<vector<int>> rotas;
    for (int i = 1; i <= n; ++i) {
        rotas.push_back({0, i, 0});
    }

    // Calculate the economies
    vector<Economia> economias;
    for (int i = 1; i <= n; ++i) {
        for (int j = i + 1; j <= n; ++j) {
            int valor = distancias[0][i] + distancias[0][j] - distancias[i][j];
            economias.push_back(Economia(i, j, valor));
        }
    }

    // Sort the economies in descending order
    sort(economias.begin(), economias.end(), compararEconomias);

    // Combine routes based on economies
    CombinaRotas(economias, rotas, distancias, demandas, maxParadas, capacidade);

    // Calculate and print the total cost
    CalcCustoTotal(rotas, distancias);
}

// Function to calculate the total cost of all routes
void CalcCustoTotal(vector<vector<int>>& rotas, const vector<vector<int>>& distancias) {
    int custoTotal = 0;
    for (const auto& rota : rotas) {
        int custoRota = calcularCustoRota(rota, distancias);
        custoTotal += custoRota;
    }
    cout << "Custo total: " << custoTotal << endl; // Print total cost
}

// Function to combine routes based on economies
void CombinaRotas(vector<Economia>& economias, vector<vector<int>>& rotas, const vector<vector<int>>& distancias, const vector<int>& demandas, int maxParadas, int capacidade) {
    for (const auto& economia : economias) {
        int i = economia.i;
        int j = economia.j;

        int rotaI = encontrarRota(i, rotas, distancias);
        int rotaJ = encontrarRota(j, rotas, distancias);

        if (rotaI != -1 && rotaJ != -1 && rotaI != rotaJ) {
            int demandaTotal = 0;
            for (int cliente : rotas[rotaI]) {
                if (cliente != 0) {
                    demandaTotal += demandas[cliente];
                }
            }
            for (int cliente : rotas[rotaJ]) {
                if (cliente != 0) {
                    demandaTotal += demandas[cliente];
                }
            }

            // Check if the combination exceeds the maximum number of stops
            int numParadasRotaI = rotas[rotaI].size() - 2; // Excluding initial and final depot
            int numParadasRotaJ = rotas[rotaJ].size() - 2; // Excluding initial and final depot
            if (numParadasRotaI + numParadasRotaJ > maxParadas) {
                continue; // Skip this combination if it exceeds the stop limit
            }

            if (demandaTotal <= capacidade) {
                // Combine the routes
                vector<int> novaRota = rotas[rotaI];
                novaRota.pop_back();
                novaRota.insert(novaRota.end(), rotas[rotaJ].begin() + 1, rotas[rotaJ].end());

                // Verify if the new route is valid
                bool rotaValida = true;
                VerificaValida(novaRota, distancias, rotaValida, rotas, rotaI, rotaJ);
            }
        }
    }
}

// Function to verify if a new route is valid
void VerificaValida(vector<int>& novaRota, const vector<vector<int>>& distancias, bool& rotaValida, vector<vector<int>>& rotas, int rotaI, int rotaJ) {
    for (size_t k = 0; k < novaRota.size() - 1; ++k) {
        if (distancias[novaRota[k]][novaRota[k + 1]] == 0) {
            rotaValida = false;
            break;
        }
    }

    if (rotaValida) {
        rotas[rotaI] = novaRota;
        rotas.erase(rotas.begin() + rotaJ);
    }
}

// Main function
int main(int argc, char* argv[]) {
    char* nomeArquivo = argv[1]; // Get the input file name from command line arguments
    int numVertices;
    vector<int> demandas = LerDestinoDemanda(nomeArquivo, numVertices); // Read demands
    vector<vector<int>> distancias = LerRotasPossiveis(nomeArquivo, numVertices); // Read distances

    int capacidade = 15; // Vehicle capacity
    int maxParadas = 5;  // Maximum stops

    auto start = high_resolution_clock::now(); // Start measuring execution time
    clarkeWright(distancias, capacidade, demandas, maxParadas - 2); // Apply Clarke-Wright algorithm
    auto end = high_resolution_clock::now(); // End measuring execution time

    auto duration = duration_cast<milliseconds>(end - start).count(); // Calculate duration
    cout << "Tempo de execução: " << duration << " ms" << endl; // Print execution time

    return 0;
}

Writing heuristica.cpp


## Execução

In [11]:
arquivo = "grafo_100_0.125.txt"

In [12]:
!g++ -g -Wall -fopenmp -o heuristica heuristica.cpp
!./heuristica $arquivo

Custo total: 5411
Tempo de execução: 42 ms


In [13]:
arquivo = "grafo_500_0.125.txt"
!./heuristica $arquivo

Custo total: 25756
Tempo de execução: 5388 ms


In [14]:
arquivo = "grafo_1000_0.125.txt"
!./heuristica $arquivo

Custo total: 49987
Tempo de execução: 36090 ms


### Slurm

In [ ]:
%%file heuristica.slurm

#!/bin/bash
#SBATCH --job-name=heuristica
#SBATCH --nodes=1
#SBATCH --partition=express
#SBATCH --mem=500M

./heuristica $arquivo

Writing heuristica.slurm


## Resultados

Pode-se dizer que a aplicação da heurística foi um grande sucesso. Ao compararmos o desemprenho da versão com heurística com a exaustiva, vemos que diferntemente da exaustiva a heurística é capaz de tornar tratáveis entradas de até mlihares de nós e executar entradas pequenas em um tempo rápido e viável para aplicações reais.

Apesar disso ao compararmos os tempos obtidos nas execuções realizadas na seção anterior, é possível ver que o crescimento da complexidade ainda continua exponencial o próximo de exponecial, ao levar 5 segundo para uma entrada de tamanho 500 e 40 para uma entrada de tamanho 1000, logo, apesar do sucesso, a solução com heurística ainda é inviável para entradas grandes, possuindo sua aplicação liitada a entradas pequenas ou médias.



# Paralelização Local

O próximo passo passo a realizarmos em nossas implementações é a tentativa de paralelização, neste caso a local, utilizando-se de múltiplos processos de uma máquina com a ajuda do openMP.

A melhor implementação possível é capaz de tornar algoritimos de tempos exponênciais em polinômiais, através da aplucação de uma estrutura semelhante a uma máquina de turing não determinística e assim, tornar grandes entradas tratáveis.

Para realizar a implementação do paralelismo para o código abaixo, optu-se por realizar apenas a parelização de fors, utilizando diferentes schedulers do openMP, sendo eles os que foram julgados como mais adequados para cada situação.

Schedulers utilizados:

Static - O scheduler statico é util para paralelizar fors com iterações que possuem mais ou menos o mesmo tempo de execução, uma vez que a divisão dos dados entre as threads é feita durante a compliação

Guided - O scheduler guiado, inicia associando grandes grupos de dados aos processos no começo da execução e menores no final, sendo útil para fors com iterações que aumentam de complexidade.

In [49]:
%%file localParallel.cpp

#include <iostream>
#include <vector>
#include <algorithm>
#include <map>
#include <climits>
#include <fstream>
#include <set>
#include <limits>
#include <chrono>
#include <omp.h>

using namespace std;
using namespace std::chrono;

// Função para ler o arquivo de entrada e retornar um vetor com as demandas
vector<int> LerDestinoDemanda(const string& nomeArquivo, int& numVertices) {
    ifstream arquivo(nomeArquivo);
    if (!arquivo.is_open()) {
        cerr << "Erro ao abrir o arquivo " << nomeArquivo << endl;
        exit(1);
    }

    arquivo >> numVertices;

    vector<int> grafo(numVertices, 0);
    int destino, demanda;
    for (int i = 1; i < numVertices; i++) {
        arquivo >> destino >> demanda;
        grafo[destino] = demanda;
    }
    arquivo.close();
    return grafo;
}

// Função para ler o arquivo de entrada e retornar um vetor com as rotas possíveis
vector<vector<int>> LerRotasPossiveis(const string& nomeArquivo, int& numVertices) {
    ifstream arquivo(nomeArquivo);
    if (!arquivo.is_open()) {
        cerr << "Erro ao abrir o arquivo " << nomeArquivo << endl;
        exit(1);
    }
    // le a primeira linha do arquivo e salva o numero de vertices
    arquivo >> numVertices;

    // ignora as proximas numVertices-1 linhas
    for (int i = 0; i < numVertices-1; i++) {
        int x, y;
        arquivo >> x >> y;
    }

    // le a quantidade de rotas possiveis
    int numRotas;
    arquivo >> numRotas;

    // cria um grafo com numVertices vertices e inicializa com 0 e le as rotas
    vector<vector<int>> grafo(numVertices, vector<int>(numVertices, 0));

    int origem, destino, custo;
    while (arquivo >> origem >> destino >> custo) {
        grafo[origem][destino] = custo;     // GRAFO DIRECIONADO!
    }
    arquivo.close();
    return grafo;
}

struct Economia {
    int i, j;
    double valor;
    Economia(int i, int j, double valor) : i(i), j(j), valor(valor) {} // Construtor
};

bool compararEconomias(const Economia& a, const Economia& b) {
    return a.valor > b.valor;
}

// Função para encontrar a rota de um nó
int encontrarRota(int node, const vector<vector<int>>& rotas, const vector<vector<int>>& distancias) {
    for (size_t i = 0; i < rotas.size(); ++i) {
        auto it = find(rotas[i].begin(), rotas[i].end(), node);
        if (it != rotas[i].end()) {
            // Verificar se a rota tem custo 0
            for (size_t j = 0; j < rotas[i].size() - 1; ++j) {
                if (distancias[rotas[i][j]][rotas[i][j + 1]] == 0) {
                    return -1; // Rota inválida
                }
            }
            return i;
        }
    }
    return -1;
}

// Função para calcular o custo de uma rota
int calcularCustoRota(const vector<int>& rota, const vector<vector<int>>& distancias) {
    int custo = 0;

    #pragma omp parallel for reduction(+:custo) schedule(static)
    for (size_t i = 0; i < rota.size() - 1; ++i) {
        custo += distancias[rota[i]][rota[i + 1]];
    }
    return custo;
}

void CombinaRotas(std::vector<Economia> &economias, std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias, const std::vector<int> &demandas, int maxParadas, int capacidade);

void VerificaValida(std::vector<int> &novaRota, const std::vector<std::vector<int>> &distancias, bool &rotaValida, std::vector<std::vector<int>> &rotas, int rotaI, int rotaJ);

void CalcCustoTotal(std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias);


void clarkeWright(const vector<vector<int>>& distancias, int capacidade, const vector<int>& demandas, int maxParadas) {
    int n = distancias.size() - 1; // número de clientes (não inclui depósito)

    // Inicializa rotas individuais
    vector<vector<int>> rotas;
    for (int i = 1; i <= n; ++i) {
        rotas.push_back({0, i, 0});
    }

    // Calcula as economias
    vector<Economia> economias;
    int numPairs = (n * (n - 1)) / 2;

    #pragma omp parallel for schedule(guided)
    for (int index = 0; index < numPairs; ++index) {
        int i = 1 + index / (n - 1);
        int j = 2 + index % (n - 1);

        if (j > n) {
            i++;
            j = i + 1;
        }

        int valor = distancias[0][i] + distancias[0][j] - distancias[i][j];

        #pragma omp critical
        economias.push_back(Economia(i, j, valor));
    }

    // Ordena as economias em ordem decrescente
    sort(economias.begin(), economias.end(), compararEconomias);

    CombinaRotas(economias, rotas, distancias, demandas, maxParadas, capacidade);

    CalcCustoTotal(rotas, distancias);
}

void CalcCustoTotal(std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias)
{
    // Calcula e imprime o custo total
    int custoTotal = 0;

    #pragma omp parallel for reduction(+:custoTotal) schedule(static)
    for (size_t i = 0; i < rotas.size(); ++i)
    {
        int custoRota = calcularCustoRota(rotas[i], distancias);
        #pragma omp critical
        {
            custoTotal += custoRota;
        }
    }
    cout << "Custo total: " << custoTotal << endl;
}

void CombinaRotas(std::vector<Economia> &economias, std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias, const std::vector<int> &demandas, int maxParadas, int capacidade)
{
    // Combina rotas com base nas economias
    for (size_t k = 0; k < economias.size(); ++k)
    {
        int i = economias[k].i;
        int j = economias[k].j;

        int rotaI = encontrarRota(i, rotas, distancias);
        int rotaJ = encontrarRota(j, rotas, distancias);

        if (rotaI != -1 && rotaJ != -1 && rotaI != rotaJ)
        {
            int demandaTotal = 0;

            for (int cliente : rotas[rotaI])
                {
                    if (cliente != 0)
                    {
                        demandaTotal += demandas[cliente];
                    }
                }
            for (int cliente : rotas[rotaJ])
                {
                    if (cliente != 0)
                    {
                        demandaTotal += demandas[cliente];
                    }
                }
            // Verificar se a combinação excede o número máximo de paradas
            int numParadasRotaI = rotas[rotaI].size() - 2; // Excluindo depósito inicial e final
            int numParadasRotaJ = rotas[rotaJ].size() - 2; // Excluindo depósito inicial e final
            if (numParadasRotaI + numParadasRotaJ > maxParadas)
            {
                continue; // Pular essa combinação se exceder o limite de paradas
            }

            if (demandaTotal <= capacidade)
            {
                // Combina as rotas
                vector<int> novaRota = rotas[rotaI];
                novaRota.pop_back();
                novaRota.insert(novaRota.end(), rotas[rotaJ].begin() + 1, rotas[rotaJ].end());

                // Verifica se a nova rota é válida
                bool rotaValida = true;
                VerificaValida(novaRota, distancias, rotaValida, rotas, rotaI, rotaJ);
            }
        }
    }
    //cout << "Rotas combinadas" << endl;
}

void VerificaValida(std::vector<int> &novaRota, const std::vector<std::vector<int>> &distancias, bool &rotaValida, std::vector<std::vector<int>> &rotas, int rotaI, int rotaJ)
{
    for (size_t k = 0; k < novaRota.size() - 1; ++k)
    {
        if (distancias[novaRota[k]][novaRota[k + 1]] == 0)
        {
            rotaValida = false;
            break;
        }
    }

    if (rotaValida)
    {
        {
            rotas[rotaI] = novaRota;
            rotas.erase(rotas.begin() + rotaJ);
        }
    }
}

int main( int argc, char *argv[] ) {
    char *nomeArquivo = argv[1];
    int numVertices;
    vector<int> demandas = LerDestinoDemanda(nomeArquivo, numVertices);
    vector<vector<int>> distancias = LerRotasPossiveis(nomeArquivo, numVertices);

    int capacidade = 15; // Capacidade do veículo
    int maxParadas = 5;

    auto start = high_resolution_clock::now();
    clarkeWright(distancias, capacidade, demandas, maxParadas - 2); // -2 pra tirar a saida e entrada
    auto end = high_resolution_clock::now();


    auto duration = duration_cast<milliseconds>(end - start).count();
    cout << "Tempo de execução: " << duration << " ms" << endl;

    return 0;
}

Overwriting localParallel.cpp


## Execução

Observações sobre a execução.

Devido a algum detalhe do colab, as vezes o algoritimo não imprime os resultados e da como completo, sem retornar erros ou qualquer sinal, caso isso ocorra, apenas execute novamente

In [50]:
arquivo = "grafo_100_0.125.txt"

In [51]:
!g++ -g -Wall -fopenmp -o localParallel localParallel.cpp

In [52]:
!./localParallel $arquivo

Custo total: 6029
Tempo de execução: 52 ms


In [53]:
arquivo = "grafo_500_0.125.txt"
!./localParallel $arquivo

Custo total: 26873
Tempo de execução: 4614 ms


In [54]:
arquivo = "grafo_1000_0.125.txt"
!./localParallel $arquivo

Custo total: 52077
Tempo de execução: 30338 ms


## Slurm

In [ ]:
%%file localParallel.slurm

#!/bin/bash
#SBATCH --job-name=localParallel
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --cores=4
#SBATCH --mem=500M

export OMP_NUM_THREADS=4
./localParallel $arquivo

Writing localParallel.slurm


## Resultados

Com base nas execuções realizadas na seção anterior podemos detectar uma melhoria no tempo de execução, em relação a aplicação apenas da heurística. é possível observar também, que a melhoria nos tempos de execução se torna cada vez mais evidente conforme o aumento das entradas, evidenciando uma melhoria no valor da exponecial.
Porém como mencionado o algoritimo ainda continua a apresentar tempo exponencial, possuindo abertura para melhorias na parelilização.

Outro ponto importante, este que deve ser melhorado em implementações futuras é a detecção de todas as "race conditions", uma vez que os resultados paralelizados foram um pouco diferentes dos resultados originais. Isto provavelmente ocorreu devido a em algum ponto da execução, dois ou mais processos acessaram a mesma variável simultanemante e por isso coletaram informações incorretas. Apesar de resultados variaveis serem um problema, é necessário notar também que em diversos casos é necessário balancear a segurança do resultado e o tempo de execução. Caso desejassemos eliminar todas as possíbilidades de conflito, poderiamos criar situações em que eliminariamos o propósito da paralelização, ao bloquear processos em pontos específicos, ou em situações piores, fazer com que o programa travesse nos pontos de concorrencia.


# Paralelização em diversas máquinas

Para a sessão abaixo, tentaremos implementar outro tipo de paralelização, ainda nos utilizando da heurístca, para este caso porém, utilizaremos do MPI para realizar compuação distribuida em diversas máquinas, ou caso deseje executar no colab ou localmente, em diversos cores da mesma máquina.

Entrando um pouco mais afundo na implementação, o MPI permite que uma máquina mestre realiza comunicação com diversas outras máquinas, enviando e recebendo informações. A capacidade de transmitir informações foi utilizada neste caso para transmitir pedações dos arrays manipulados ao longo do código, com objetivo de distribuir o trabalho entre as máquinas.

In [ ]:
%%file mpiV2.cpp

#include <iostream>
#include <vector>
#include <algorithm>
#include <map>
#include <climits>
#include <fstream>
#include <set>
#include <limits>
#include <chrono>
#include <mpi.h>

using namespace std;
using namespace std::chrono;

// Function declarations
vector<int> LerDestinoDemanda(const string& nomeArquivo, int& numVertices);
vector<vector<int>> LerRotasPossiveis(const string& nomeArquivo, int& numVertices);
struct Economia;
bool compararEconomias(const Economia& a, const Economia& b);
int encontrarRota(int node, const vector<vector<int>>& rotas, const vector<vector<int>>& distancias);
int calcularCustoRota(const vector<int>& rota, const vector<vector<int>>& distancias);
void CombinaRotas(std::vector<Economia> &economias, std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias, const std::vector<int> &demandas, int maxParadas, int capacidade, int rank, int size);
void VerificaValida(std::vector<int> &novaRota, const std::vector<std::vector<int>> &distancias, bool &rotaValida, std::vector<std::vector<int>> &rotas, int rotaI, int rotaJ);
void CalcCustoTotal(std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias);

struct Economia {
    int i, j;
    double valor;
    Economia() = default;
    Economia(int i, int j, double valor) : i(i), j(j), valor(valor) {}
};

bool compararEconomias(const Economia& a, const Economia& b) {
    return a.valor > b.valor;
}

int encontrarRota(int node, const vector<vector<int>>& rotas, const vector<vector<int>>& distancias) {
    for (size_t i = 0; i < rotas.size(); ++i) {
        auto it = find(rotas[i].begin(), rotas[i].end(), node);
        if (it != rotas[i].end()) {
            return i;
        }
    }
    return -1;
}

int calcularCustoRota(const vector<int>& rota, const vector<vector<int>>& distancias) {
    int custo = 0;
    for (size_t i = 0; i < rota.size() - 1; ++i) {
        custo += distancias[rota[i]][rota[i + 1]];
    }
    return custo;
}

void CombinaRotas(std::vector<Economia> &economias, std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias, const std::vector<int> &demandas, int maxParadas, int capacidade, int rank, int size) {
    vector<Economia> allEconomias;

    // Gather all local economias at rank 0
    if (rank == 0) {
        allEconomias.insert(allEconomias.end(), economias.begin(), economias.end());
        for (int i = 1; i < size; ++i) {
            int numLocalEconomias;
            MPI_Status status;
            MPI_Probe(i, 0, MPI_COMM_WORLD, &status);
            MPI_Get_count(&status, MPI_BYTE, &numLocalEconomias);
            vector<Economia> localEconomias(numLocalEconomias / sizeof(Economia));
            MPI_Recv(localEconomias.data(), numLocalEconomias, MPI_BYTE, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            allEconomias.insert(allEconomias.end(), localEconomias.begin(), localEconomias.end());
        }
        sort(allEconomias.begin(), allEconomias.end(), compararEconomias);
    } else {
        MPI_Send(economias.data(), economias.size() * sizeof(Economia), MPI_BYTE, 0, 0, MPI_COMM_WORLD);
    }

    // Only rank 0 performs the route combination
    if (rank == 0) {
        for (const auto &economia : allEconomias) {
            int i = economia.i;
            int j = economia.j;

            int rotaI = encontrarRota(i, rotas, distancias);
            int rotaJ = encontrarRota(j, rotas, distancias);

            if (rotaI != -1 && rotaJ != -1 && rotaI != rotaJ) {
                int demandaTotal = 0;
                for (int cliente : rotas[rotaI]) {
                    if (cliente != 0) {
                        demandaTotal += demandas[cliente];
                    }
                }
                for (int cliente : rotas[rotaJ]) {
                    if (cliente != 0) {
                        demandaTotal += demandas[cliente];
                    }
                }

                int numParadasRotaI = rotas[rotaI].size() - 2;
                int numParadasRotaJ = rotas[rotaJ].size() - 2;
                if (numParadasRotaI + numParadasRotaJ > maxParadas) {
                    continue;
                }

                if (demandaTotal <= capacidade) {
                    vector<int> novaRota = rotas[rotaI];
                    novaRota.pop_back();
                    novaRota.insert(novaRota.end(), rotas[rotaJ].begin() + 1, rotas[rotaJ].end());

                    bool rotaValida = true;
                    VerificaValida(novaRota, distancias, rotaValida, rotas, rotaI, rotaJ);
                }
            }
        }
    }

    // Broadcast the updated routes to all processes
    int numRotas = rotas.size();
    MPI_Bcast(&numRotas, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (rank != 0) {
        rotas.resize(numRotas);
    }

    for (int i = 0; i < numRotas; ++i) {
        int rotaSize = rotas[i].size();
        MPI_Bcast(&rotaSize, 1, MPI_INT, 0, MPI_COMM_WORLD);

        if (rank != 0) {
            rotas[i].resize(rotaSize);
        }

        MPI_Bcast(rotas[i].data(), rotaSize, MPI_INT, 0, MPI_COMM_WORLD);
    }
}

void VerificaValida(std::vector<int> &novaRota, const std::vector<std::vector<int>> &distancias, bool &rotaValida, std::vector<std::vector<int>> &rotas, int rotaI, int rotaJ) {
    for (size_t k = 0; k < novaRota.size() - 1; ++k) {
        if (distancias[novaRota[k]][novaRota[k + 1]] == 0) {
            rotaValida = false;
            break;
        }
    }

    if (rotaValida) {
        rotas[rotaI] = novaRota;
        rotas.erase(rotas.begin() + rotaJ);
    }
}

void CalcCustoTotal(std::vector<std::vector<int>> &rotas, const std::vector<std::vector<int>> &distancias) {
    int custoTotal = 0;
    for (const auto &rota : rotas) {
        int custoRota = calcularCustoRota(rota, distancias);
        custoTotal += custoRota;
    }
    cout << "Custo total: " << custoTotal << endl;
}

vector<int> LerDestinoDemanda(const string& nomeArquivo, int& numVertices) {
    ifstream arquivo(nomeArquivo);
    if (!arquivo.is_open()) {
        cerr << "Erro ao abrir o arquivo " << nomeArquivo << endl;
        exit(1);
    }

    arquivo >> numVertices;

    vector<int> grafo(numVertices, 0);
    int destino, demanda;
    for (int i = 1; i < numVertices; i++) {
        arquivo >> destino >> demanda;
        grafo[destino] = demanda;
    }
    arquivo.close();
    return grafo;
}

vector<vector<int>> LerRotasPossiveis(const string& nomeArquivo, int& numVertices) {
    ifstream arquivo(nomeArquivo);
    if (!arquivo.is_open()) {
        cerr << "Erro ao abrir o arquivo " << nomeArquivo << endl;
        exit(1);
    }
    arquivo >> numVertices;

    for (int i = 0; i < numVertices-1; i++) {
        int x, y;
        arquivo >> x >> y;
    }

    int numRotas;
    arquivo >> numRotas;

    vector<vector<int>> grafo(numVertices, vector<int>(numVertices, 0));

    int origem, destino, custo;
    while (arquivo >> origem >> destino >> custo) {
        grafo[origem][destino] = custo;
    }
    arquivo.close();
    return grafo;
}

int main(int argc, char* argv[]) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int numVertices;
    vector<int> demandas;
    vector<vector<int>> distancias;

    if (rank == 0) {
        char* nomeArquivo = argv[1];
        demandas = LerDestinoDemanda(nomeArquivo, numVertices);
        distancias = LerRotasPossiveis(nomeArquivo, numVertices);
    }

    MPI_Bcast(&numVertices, 1, MPI_INT, 0, MPI_COMM_WORLD);

    if (rank != 0) {
        demandas.resize(numVertices);
        distancias.resize(numVertices, vector<int>(numVertices));
    }

    MPI_Bcast(demandas.data(), numVertices, MPI_INT, 0, MPI_COMM_WORLD);

    for (int i = 0; i < numVertices; ++i) {
        MPI_Bcast(distancias[i].data(), numVertices, MPI_INT, 0, MPI_COMM_WORLD);
    }

    int capacidade = 15;
    int maxParadas = 5;

    auto start = high_resolution_clock::now();

    vector<vector<int>> rotas;
    for (int i = 1; i < numVertices; ++i) {
        rotas.push_back({0, i, 0});
    }

    vector<Economia> economias;
    for (int i = 1; i < numVertices; ++i) {
        for (int j = i + 1; j < numVertices; ++j) {
            int valor = distancias[0][i] + distancias[0][j] - distancias[i][j];
            economias.push_back(Economia(i, j, valor));
        }
    }

    int economiasPerProcess = economias.size() / size;
    int remainder = economias.size() % size;
    vector<Economia> localEconomias;

    if (rank == 0) {
        for (int i = 0; i < size; ++i) {
            int startIdx = i * economiasPerProcess + min(i, remainder);
            int endIdx = startIdx + economiasPerProcess + (i < remainder ? 1 : 0);
            if (i == 0) {
                localEconomias.insert(localEconomias.end(), economias.begin() + startIdx, economias.begin() + endIdx);
            } else {
                MPI_Send(economias.data() + startIdx, (endIdx - startIdx) * sizeof(Economia), MPI_BYTE, i, 0, MPI_COMM_WORLD);
            }
        }
    } else {
        int numLocalEconomias;
        MPI_Status status;
        MPI_Probe(0, 0, MPI_COMM_WORLD, &status);
        MPI_Get_count(&status, MPI_BYTE, &numLocalEconomias);
        localEconomias.resize(numLocalEconomias / sizeof(Economia));
        MPI_Recv(localEconomias.data(), numLocalEconomias, MPI_BYTE, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    }

    sort(localEconomias.begin(), localEconomias.end(), compararEconomias);

    CombinaRotas(localEconomias, rotas, distancias, demandas, maxParadas, capacidade, rank, size);

    if (rank == 0) {
        CalcCustoTotal(rotas, distancias);
    }

    auto end = high_resolution_clock::now();
    cout << "Rank " << rank << " finished" << endl;
    auto duration = duration_cast<milliseconds>(end - start).count();
    if (rank == 0) {
        cout << "Tempo de execução: " << duration << " ms" << endl;
    }

    MPI_Finalize();
    return 0;
}

Overwriting mpiV2.cpp


In [ ]:
!mpic++ -g -Wall -o mpiV2 mpiV2.cpp

In [ ]:
arquivo = "grafo_100_0.125.txt"
!mpirun --allow-run-as-root --oversubscribe -n 2 ./mpiV2 $arquivo

Custo total: 5796
Rank 1 finished
Rank 0 finished
Tempo de execução: 117 ms


In [ ]:
arquivo = "grafo_500_0.125.txt"
!mpirun --allow-run-as-root --oversubscribe -n 2 ./mpiV2 $arquivo

Rank 1 finished
Custo total: 25517
Rank 0 finished
Tempo de execução: 6503 ms


In [ ]:
arquivo = "grafo_1000_0.125.txt"
!mpirun --allow-run-as-root --oversubscribe -n 2 ./mpiV2 $arquivo

Rank 1 finished
Custo total: 48742
Rank 0 finished
Tempo de execução: 54629 ms


In [ ]:
arquivo = "grafo_100_0.125.txt"
!mpirun --allow-run-as-root --oversubscribe -n 4 ./mpiV2 $arquivo

Rank 1 finished
Rank 2 finished
Rank 3 finished
Custo total: 5685
Rank 0 finished
Tempo de execução: 117 ms


In [ ]:
arquivo = "grafo_500_0.125.txt"
!mpirun --allow-run-as-root --oversubscribe -n 4 ./mpiV2 $arquivo

Rank 2 finished
Rank 3 finished
Rank 1 finished
Custo total: 25824
Rank 0 finished
Tempo de execução: 13785 ms


In [ ]:
arquivo = "grafo_1000_0.125.txt"
!mpirun --allow-run-as-root --oversubscribe -n 4 ./mpiV2 $arquivo

Rank 1 finished
Rank 2 finished
Rank 3 finished
Custo total: 49661
Rank 0 finished
Tempo de execução: 105807 ms


## Slurm

In [ ]:
%%file global_parallel.slurm

#!/bin/bash
#SBATCH --job-name=global_parallel
#SBATCH --nodes=2
#SBATCH --mem=500M
#SBATCH --time=00:02:00
#SBATCH --partition=espec

mpirun ./global_parallel $arquivo

Writing global_parallel.slurm


## Resultados


### Analise com os dados do cluster

#### Dados (obtidos no cluster da disciplina durante a tarde de 03/06/24 e manhã de 05/06/24 utilizando o slurm acima)

- Heuristica:

100 nós:
Custo total: 6431
Tempo de execução: 27 ms

500 nós:
Custo total: 26444
Tempo de execução: 2625 ms

1000 nós:
Custo total: 50208
Tempo de execução: 20230 ms

- MPI+Heurística:

100 nós:
Custo total: 5953
Tempo de execução: 13 ms

500 nós:
Custo total: 25249
Tempo de execução: 1257 ms

1000 nós:
Custo total: 48030
Tempo de execução: 9719 ms


Como é possívele visualizar através dos dados obtidos, a implementação do MPI resultou em uma melhoria significativa no tempo de execução, sendo possível observar uma redução de tempo de execução de até, aproximadamente, 50% em relação a execução da heurística pura.

Apesar disso, é possível observar uma pequena diferença entre os resultados obtidos, sendo que a execução do MPI resultou em um custo total um pouco menor que a execução da heurística pura, oque pode ser explicado pela natureza da paralelização, uma vez que o conjunto de dados é divido entre as máquinas e caso o vetor de dados não seja um número divisível pelo número de máquinas, pode ocorrer que algumas máquinas recebam mais dados que outras, resultando em resultados levemente diferentes.

Além disso, durante a execução do código em MPI podem ocorrer pequenas variações nos resultados, devido a ordem de execução de operações matemáticas que poem variar de execução para execução, além de problemas de precisão decimal, latência e problemas durante transmissões de dados, que acumuladas podem resultar em impactos mais significativos nos resultados.


### Analise com os dados do colab

Como é possível observar na seção de execução acima, apesar dos melhores esforços não foi possível obter uma melhoria em relaçõa ao tempo de execução ao compararmos a implementação da heurística pura, com a com adição de MPI, sendo possível notar situações onde inclusive ocorreu piora do tempo.

Isso pode ser devido a que no colab a paralelização esta sendo feita no local, ou seja, em diferentes cores da mesma máquina, oque pode resultar em um overhead de comunicação maior que o benefício da paralelização, uma vez que a comunicação entre os processos é feita de maneira mais lenta que a execução do código em si.